In [18]:
import sqlite3
import numpy as np
import pandas as pd
from jdcal import gcal2jd, jd2gcal, jcal2jd, MJD_0
import keras
import math
import tensorflowjs as tfjs
import pickle
import julian as julian
import datetime
from keras.models import Sequential, model_from_yaml, load_model
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [11]:
def encodeOutputVariable(y):
    labelencoder_Y_Origin = LabelEncoder()
    y = labelencoder_Y_Origin.fit_transform(y.astype(str))
    return y

def encodeCategoricalData(X, index):
    # encode categorical data
    labelencoder_X_Origin = LabelEncoder()
    X[:, index] = labelencoder_X_Origin.fit_transform(X[:, index].astype(str))
    return X    

def manualEncodeLongStrings(X, column):
    index = 0
    test = 0
    keys = {}
    for row in X:
        key = row[column].replace(", ", "").replace(" ", "")
        if (keys.get(key) == None):
            keys[key] = index
            index += 1
        X[test][column] = keys.get(key)
        test += 1
    return X
    
def defaultMinimumValues(values): 
    for index, x in enumerate(values):
        x = [float(y) if y != None and float(y) > 0.0 else 0 for y in x]
        values[index] = x
    return pd.DataFrame(values)

def convertDateColumns(X, column):
    X = X[:, column].apply(lambda row: julian.from_jd(row, fmt='mmddyyyy'))
    print(X[:, column])
    return X.values

def encodeHotEncoder(X, categoryIndex):
    # meant to create dummy variables for each category data
    # you only use it for one column at a time, output will be the number of columns
    # needed to represent all discrete values of column
    onehotencoder = OneHotEncoder(categorical_features = [categoryIndex])
    X = defaultMinimumValues(X)
    X = onehotencoder.fit_transform(X.astype(str)).toarray()    
    X = X[:, 1:]
    return X

def determineTotalTime(startDay, startTime, endDay, endTime):
    minsPerDay = 24 * 60
    totalStart = (int(startDay) * minsPerDay) + int(startTime)
    totalEnd = (int(endDay) * minsPerDay) + int(endTime)
    return totalEnd - totalStart

In [12]:
sqlite_file = "./wildfires.sqlite"

# connecting to the database file and saving the select
conn = sqlite3.connect(sqlite_file)
dataset = pd.read_sql_query("select * from Fires limit 50000;", conn)

In [13]:
dataset.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


In [14]:
# split dataset into train and test lists
X = dataset.iloc[:, [34, 30, 31, 23, 20, 25]]
y = dataset.iloc[:, 29].values

In [15]:
X.head()

,STATE,LATITUDE,LONGITUDE,STAT_CAUSE_CODE,DISCOVERY_DATE,CONT_DATE
0,CA,40.036944,-121.005833,9.0,2453403.5,2453403.5
1,CA,38.933056,-120.404444,1.0,2453137.5,2453137.5
2,CA,38.984167,-120.735556,5.0,2453156.5,2453156.5
3,CA,38.559167,-119.913333,1.0,2453184.5,2453189.5
4,CA,38.559167,-119.933056,1.0,2453184.5,2453189.5


In [16]:
test = X.iloc[:, 4].map(lambda row: jd2gcal(*jcal2jd(row)))

TypeError: jcal2jd() missing 2 required positional arguments: 'month' and 'day'

In [70]:
test.head()

0   1970-01-01 00:00:00.002453403
1   1970-01-01 00:00:00.002453137
2   1970-01-01 00:00:00.002453156
3   1970-01-01 00:00:00.002453184
4   1970-01-01 00:00:00.002453184
Name: DISCOVERY_DATE, dtype: datetime64[ns]

In [17]:
mjd = 54372.78
dt = julian.from_jd(mjd, fmt='mjd')
print(dt)

2007-09-29 18:43:11.999982


In [24]:
mjd = 53403.5
dt = julian.from_jd(mjd, fmt='jd')
print(dt)

ValueError: year -4566 is out of range